# Project 2 - Task 1 Notebook


In [14]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import warnings
import torch
from ultralytics import YOLO
from transformers import pipeline
from torchvision.datasets import VOCDetection
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from transformers import RTDetrForObjectDetection, RTDetrImageProcessor
from transformers import DetrForObjectDetection, DetrImageProcessor

In [15]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)


set_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [16]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print("Using device", device)

Using device cuda:0


In [17]:
dataset = VOCDetection('data/', year="2007", image_set="val", download=False)
subset_size = 1000
subset = [dataset[i] for i in range(subset_size)]

# Mapping VOC classes to IDs
VOC_CLASSES = [
    "aeroplane", "bicycle", "bird", "boat", "bottle",
    "bus", "car", "cat", "chair", "cow",
    "diningtable", "dog", "horse", "motorbike", "person",
    "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]
VOC_CLASS_TO_ID = {cls_name: idx + 1 for idx, cls_name in enumerate(VOC_CLASSES)}

In [18]:
# -----------------------------
# Prepare targets in TorchMetrics format
# -----------------------------
targets_list = []
for img, target in subset:
    objs = target['annotation']['object']
    if not isinstance(objs, list):
        objs = [objs]
    boxes = []
    labels = []
    for obj in objs:
        b = obj['bndbox']
        boxes.append([int(b['xmin']), int(b['ymin']), int(b['xmax']), int(b['ymax'])])
        labels.append(VOC_CLASS_TO_ID[obj['name']])
    targets_list.append({
        "boxes": torch.tensor(boxes, dtype=torch.float32),
        "labels": torch.tensor(labels, dtype=torch.int64)
    })
targets_list

[{'boxes': tensor([[263., 211., 324., 339.],
          [165., 264., 253., 372.],
          [  5., 244.,  67., 374.],
          [241., 194., 295., 299.],
          [277., 186., 312., 220.]]),
  'labels': tensor([9, 9, 9, 9, 9])},
 {'boxes': tensor([[141.,  50., 500., 330.]]), 'labels': tensor([7])},
 {'boxes': tensor([[ 69., 172., 270., 330.],
          [150., 141., 229., 284.],
          [285., 201., 327., 331.],
          [258., 198., 297., 329.]]),
  'labels': tensor([13, 15, 15, 15])},
 {'boxes': tensor([[ 92.,  72., 305., 473.]]), 'labels': tensor([2])},
 {'boxes': tensor([[231.,  88., 483., 256.],
          [ 11., 113., 266., 259.]]),
  'labels': tensor([8, 8])},
 {'boxes': tensor([[ 33., 148., 371., 416.]]), 'labels': tensor([7])},
 {'boxes': tensor([[  1., 235., 182., 388.],
          [210.,  36., 336., 482.],
          [ 46.,  82., 170., 365.],
          [ 11., 181., 142., 419.]]),
  'labels': tensor([12, 15, 15, 15])},
 {'boxes': tensor([[196., 165., 489., 247.]]), 'labels': t

In [19]:
def convert_yolo_predictions(yolo_results, coco_classes, voc_map):
    yolo_boxes = torch.tensor(yolo_results.boxes.xyxy, dtype=torch.float32)
    yolo_scores = torch.tensor(yolo_results.boxes.conf, dtype=torch.float32)

    # Convert YOLO class indices → names → VOC ids
    labels_list = []
    for label_idx in yolo_results.boxes.cls:
        name = coco_classes[label_idx.item()]
        label_id = voc_map.get(name, 0)
        labels_list.append(label_id)

    yolo_labels = torch.tensor(labels_list, dtype=torch.int64)

    return {
        "boxes": yolo_boxes,
        "scores": yolo_scores,
        "labels": yolo_labels,
    }


def convert_detr_predictions(detr_results, coco_classes, voc_map):
    #detr_results = [r for r in detr_results if r["score"] >= confidence]

    labels_list = []
    for l in detr_results["labels"]:
        name = coco_classes[l.item()]
        label_id = voc_map.get(name, 0)
        labels_list.append(label_id)

    return {
        "boxes": detr_results['boxes'],
        "scores": detr_results['scores'],
        "labels": torch.tensor(labels_list, dtype=torch.int64)
    }


def convert_rtdetr_predictions(rtdetr_results, coco_classes, voc_map):
    labels_list = []
    for l in rtdetr_results["labels"]:
        name = coco_classes[l.item()]
        label_id = voc_map.get(name, 0)
        labels_list.append(label_id)

    return {
        "boxes": rtdetr_results['boxes'],
        "scores": rtdetr_results['scores'],
        "labels": torch.tensor(labels_list, dtype=torch.int64)
    }

In [20]:
def evaluate_yolo(yolo_model, subset, targets_list, voc_map, conf):
    metric_yolo = MeanAveragePrecision()

    for sample, target in zip(subset, targets_list):
        img = sample[0]
        img_np = np.array(img)
        yolo_raw = yolo_model(img_np, conf=conf, verbose=False)[0]
        yolo_preds = convert_yolo_predictions(yolo_raw, yolo_model.names, voc_map)
        metric_yolo.update([yolo_preds], [target])

    return metric_yolo.compute()


def evaluate_detr(detr_processor, detr_model, subset, targets_list, voc_map, conf):
    metric_detr = MeanAveragePrecision()

    for sample, target in zip(subset, targets_list):
        img = sample[0]
        img_np = np.array(img)

        inputs = detr_processor(images=img_np, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = detr_model(**inputs)

        height, width = img_np.shape[:2]

        detr_raw = detr_processor.post_process_object_detection(
            outputs, target_sizes=[(height, width)], threshold=conf)[0]

        detr_preds = convert_detr_predictions(detr_raw, detr_model.config.id2label, voc_map)

        metric_detr.update([detr_preds], [target])

    return metric_detr.compute()


def evaluate_rtdetr(rtdetr_image_processor, rtdetr_model, subset, targets_list, voc_map, conf):
    metric_rtdetr = MeanAveragePrecision()

    for sample, target in zip(subset, targets_list):
        img = sample[0]
        img_np = np.array(img)

        inputs = rtdetr_image_processor(img_np, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = rtdetr_model(**inputs)

        height, width = img_np.shape[:2]
        rtdetr_raw = rtdetr_image_processor.post_process_object_detection(
            outputs, target_sizes=[(height, width)], threshold=conf
        )[0]
        rtdetr_preds = convert_rtdetr_predictions(rtdetr_raw,
                                                  rtdetr_model.config.id2label, voc_map)
        metric_rtdetr.update([rtdetr_preds], [target])

    return metric_rtdetr.compute()

In [21]:
def print_metrics(results, confidence):
    for conf in confidence:
        print(f"Confidence Threshold: {conf} \n")

        for model_name in ['yolo', 'detr', 'rt-detr']:
            result = results[model_name][conf]

            print(f"\n{model_name.upper()} Metrics:")
            print(f"  mAP :  {result['map']:.4f}")
            print(f"  mAP@50:    {result['map_50']:.4f}")
            print(f"  mAP@75 :    {result['map_75']:.4f}")
            print(f"  mAP (small objects):  {result['map_small']:.4f}")
            print(f"  mAP (medium objects): {result['map_medium']:.4f}")
            print(f"  mAP (large objects):  {result['map_large']:.4f}")
            print(f"  mAR@1:                {result['mar_1']:.4f}")
            print(f"  mAR@10:               {result['mar_10']:.4f}")
            print(f"  mAR@100:              {result['mar_100']:.4f}")
            print(f"  mAR (small objects):  {result['mar_small']:.4f}")
            print(f"  mAR (medium objects): {result['mar_medium']:.4f}")
            print(f"  mAR (large objects):  {result['mar_large']:.4f}")

In [22]:
yolo_model = YOLO("yolo11s.pt").to(device)

detr_processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
detr_model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50").to(device)

rtdetr_image_processor = RTDetrImageProcessor.from_pretrained("PekingU/rtdetr_r50vd")
rtdetr_model = RTDetrForObjectDetection.from_pretrained("PekingU/rtdetr_r50vd").to(device)


def evaluate_models(subset, targets_list, voc_map, confidence):
    results = {
        'yolo': {},
        'detr': {},
        'rt-detr': {}
    }

    for conf in confidence:
        results['yolo'][conf] = evaluate_yolo(yolo_model, subset, targets_list, voc_map, conf)
        results['detr'][conf] = evaluate_detr(detr_processor, detr_model, subset, targets_list, voc_map, conf)
        results['rt-detr'][conf] = evaluate_rtdetr(rtdetr_image_processor, rtdetr_model,
                                                   subset, targets_list, voc_map, conf)
    return results



Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
confidence = [0.001, 0.05, 0.1, 0.3, 0.5]
results = evaluate_models(subset, targets_list, VOC_CLASS_TO_ID, confidence)
#print_metrics(results, confidence)

/tmp/ipykernel_48738/1997881637.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yolo_boxes = torch.tensor(yolo_results.boxes.xyxy, dtype=torch.float32)
/tmp/ipykernel_48738/1997881637.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yolo_scores = torch.tensor(yolo_results.boxes.conf, dtype=torch.float32)


Confidence Threshold: 0.001 


YOLO Metrics:
  mAP :  0.4485
  mAP@50:    0.5877
  mAP@75 :    0.4880
  mAP (small objects):  0.1928
  mAP (medium objects): 0.3729
  mAP (large objects):  0.5402
  mAR@1:                0.3087
  mAR@10:               0.4940
  mAR@100:              0.5177
  mAR (small objects):  0.3585
  mAR (medium objects): 0.4784
  mAR (large objects):  0.5841

DETR Metrics:
  mAP :  0.4385
  mAP@50:    0.5962
  mAP@75 :    0.4738
  mAP (small objects):  0.2114
  mAP (medium objects): 0.3683
  mAP (large objects):  0.5458
  mAR@1:                0.3187
  mAR@10:               0.5024
  mAR@100:              0.5285
  mAR (small objects):  0.3163
  mAR (medium objects): 0.4734
  mAR (large objects):  0.6105

RT-DETR Metrics:
  mAP :  0.6767
  mAP@50:    0.8817
  mAP@75 :    0.7248
  mAP (small objects):  0.3058
  mAP (medium objects): 0.5470
  mAP (large objects):  0.7982
  mAR@1:                0.4920
  mAR@10:               0.7572
  mAR@100:              0.7992
  mAR (

In [24]:
print_metrics(results, confidence)

Confidence Threshold: 0.001 


YOLO Metrics:
  mAP :  0.4485
  mAP@50:    0.5877
  mAP@75 :    0.4880
  mAP (small objects):  0.1928
  mAP (medium objects): 0.3729
  mAP (large objects):  0.5402
  mAR@1:                0.3087
  mAR@10:               0.4940
  mAR@100:              0.5177
  mAR (small objects):  0.3585
  mAR (medium objects): 0.4784
  mAR (large objects):  0.5841

DETR Metrics:
  mAP :  0.4385
  mAP@50:    0.5962
  mAP@75 :    0.4738
  mAP (small objects):  0.2114
  mAP (medium objects): 0.3683
  mAP (large objects):  0.5458
  mAR@1:                0.3187
  mAR@10:               0.5024
  mAR@100:              0.5285
  mAR (small objects):  0.3163
  mAR (medium objects): 0.4734
  mAR (large objects):  0.6105

RT-DETR Metrics:
  mAP :  0.6767
  mAP@50:    0.8817
  mAP@75 :    0.7248
  mAP (small objects):  0.3058
  mAP (medium objects): 0.5470
  mAP (large objects):  0.7982
  mAR@1:                0.4920
  mAR@10:               0.7572
  mAR@100:              0.7992
  mAR (